In [46]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [47]:
!pip install captum
!pip install fvcore

In [48]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from captum.attr import visualization as viz
from fvcore.nn import FlopCountAnalysis, flop_count_table
from captum.attr import IntegratedGradients, LayerConductance, DeepLift, LayerDeepLift,LayerIntegratedGradients
import itertools
import pandas as pd
import os
import random
import numpy as np
from itertools import product
from torch.utils.data import SequentialSampler, RandomSampler
from torch.utils.data import DataLoader

In [49]:
method_names = ["LayerIntegratedGradients", "LayerDeepLift"]
INPUT_SHAPE= (1, 1, 28, 28)
Training_Device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
Feature_Attribution_Device = torch.device("cpu")
NUM_EPOCHS=1
NUM_CLASSES=10
IN_CHANNELS=1
NUM_RUN=15

# Model structure defination

In [50]:
class Inception(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(Inception, self).__init__()

        # First branch with 3x3 convolution
        self.branch1 = nn.Sequential(
            nn.Conv2d(in_channels, 16, kernel_size=3, padding=1),
            nn.ReLU()
        )

        # Second branch with 3x3 convolution
        self.branch2 = nn.Sequential(
            nn.Conv2d(in_channels, 16, kernel_size=3, padding=1),
            nn.ReLU()
        )

        # Third branch with 3x3 convolution
        self.branch3 = nn.Sequential(
            nn.Conv2d(in_channels, 16, kernel_size=3, padding=1),
            nn.ReLU()
        )

        # The size for the linear layer needs to be calculated based on the output dimensions of the concatenated tensors
        # For example, for 28x28 input images (as in MNIST), the size might be different
        self.fc = nn.Sequential(
            nn.Linear(16 * 3 * INPUT_SHAPE[-1] * INPUT_SHAPE[-2], 256),  # Replace <height> and <width> with the dimensions after the convolutional layers
            nn.ReLU(),
            nn.Linear(256, num_classes)  # Number of output classes
        )

    def forward(self, x):
        x.requires_grad_(True)
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        x3 = self.branch3(x)
        x = torch.cat((x1, x2, x3), dim=1)  # Concatenating along the channel dimension
        x = x.view(x.size(0), -1)  # Flatten the tensor for the fully connected layer
        x = self.fc(x)
        return x

# Get dataset

In [51]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = datasets.FashionMNIST('.', train=True, download=True, transform=transform)
test_dataset = datasets.FashionMNIST('.', train=False, download=True, transform=transform)

# FLOP Count

In [52]:
def count_flops(model, input_size):
    inputs = torch.randn(input_size)
    flops = FlopCountAnalysis(model, inputs)
    return flop_count_table(flops)

inception_model = Inception(1, 10)
# Assuming the input size for CIFAR-10 (batch size, channels, height, width)
input_size = (1, 1, 28, 28)
flops_table = count_flops(inception_model, input_size)
print(flops_table)

| module             | #parameters or shape   | #flops   |
|:-------------------|:-----------------------|:---------|
| model              | 9.637M                 | 9.975M   |
|  branch1.0         |  0.16K                 |  0.113M  |
|   branch1.0.weight |   (16, 1, 3, 3)        |          |
|   branch1.0.bias   |   (16,)                |          |
|  branch2.0         |  0.16K                 |  0.113M  |
|   branch2.0.weight |   (16, 1, 3, 3)        |          |
|   branch2.0.bias   |   (16,)                |          |
|  branch3.0         |  0.16K                 |  0.113M  |
|   branch3.0.weight |   (16, 1, 3, 3)        |          |
|   branch3.0.bias   |   (16,)                |          |
|  fc                |  9.637M                |  9.636M  |
|   fc.0             |   9.634M               |   9.634M |
|    fc.0.weight     |    (256, 37632)        |          |
|    fc.0.bias       |    (256,)              |          |
|   fc.2             |   2.57K                |   2.56K 

# Train and attribution functions

train and eval function

In [53]:
def train(epoch, model, train_loader, optimizer, criterion, device):
    model.train()
    train_loss = 0
    correct = 0
    total = 0

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = output.max(1)
        total += target.size(0)
        correct += predicted.eq(target).sum().item()

    train_accuracy = 100. * correct / total
    print(f"Epoch {epoch}: Train Loss = {train_loss / len(train_loader):.4f}, Train Accuracy = {train_accuracy:.2f}%")

    return train_accuracy

def test(epoch, model, test_loader, criterion, device):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            test_loss += loss.item()
            _, predicted = output.max(1)
            total += target.size(0)
            correct += predicted.eq(target).sum().item()

    test_accuracy = 100. * correct / total
    print(f"Epoch {epoch}: Test Loss = {test_loss / len(test_loader):.4f}, Test Accuracy = {test_accuracy:.2f}%")

    return test_accuracy


functions for calculate attribution

In [54]:
def print_ig(test_loader, model, device):
    # Move the model to the device (CPU or CUDA)
    model.to(device)

    # Ensure the model is in evaluation mode
    model.eval()

    # Get a single batch from the test loader
    inputs, target_class = next(iter(test_loader))
    inputs = inputs.to(device)

    ig_attributions = {}

    # Iterate through each named module and compute attributions for Conv2d layers with learnable parameters
    for layer_name, layer_module in model.named_modules():
        # Check if the layer is a Conv2d layer with learnable parameters
        if isinstance(layer_module, nn.Conv2d) and any(p.requires_grad for p in layer_module.parameters(recurse=False)):
            # Initialize LayerIntegratedGradients for the layer
            lig = LayerIntegratedGradients(model, layer_module)

            # Compute the attributions for the current layer
            try:
                attributions = lig.attribute(inputs, target=target_class.to(device))
            except Exception as e:
                print(f"Error computing attributions for layer {layer_name}: {e}")
                continue

            # Print out the attributions for the current layer
            print(f'Layer: {layer_name}')
            print(f'Attribution: {attributions.cpu().detach().numpy().sum()}')

            # Store the sum of attributions in the dictionary
            ig_attributions[layer_name] = attributions.cpu().detach().numpy().sum()

            # Free up memory
            del attributions, lig

    return ig_attributions

# Usage example:
# ig_attributions = print_ig(test_loader, model, device)


In [55]:
def print_deeplift(test_loader, model, device):
    # Move the model to the specified device and set it to evaluation mode
    model.to(device).eval()

    # Get a batch of data from the loader
    inputs, target_class = next(iter(test_loader))
    inputs, target_class = inputs.to(device), target_class.to(device)

    dl_attributions = {}

    # Now compute the attributions for Conv2d layers
    for layer_name, layer_module in model.named_modules():
        # Skip the whole model's container and focus on Conv2d layers with learnable parameters
        if isinstance(layer_module, nn.Conv2d) and any(p.requires_grad for p in layer_module.parameters(recurse=False)):
            # Initialize LayerDeepLift with the current layer
            ldl = LayerDeepLift(model, layer_module)

            # Compute the attributions for the current layer
            '''try:
                attributions_ldl = ldl.attribute(inputs, target=target_class.to(device))
            except Exception as e:
                print(f"Error computing attributions for layer {layer_name}: {e}")
                continue'''
            attributions_ldl = ldl.attribute(inputs, target=target_class.to(device))


            # Print out the attributions for the current layer
            print(f'Layer: {layer_name}')
            print(attributions_ldl.cpu().data.numpy().sum())

            dl_attributions[layer_name] = attributions_ldl.cpu().data.numpy().sum()

            del attributions_ldl, ldl

    return dl_attributions

# Usage example:
# dl_attributions = print_deeplift(test_loader, model, device)


# Possible Hyperparameter grid search creation

in hyperparams_list_dict, each hyperparameter has corresponding possible choices as a list, during experiment, given hyperparams sequence, location each hyperperameter's location using hyperparameter encoding function to convert strings or classes back into their index in hyperparams_list_dict

In [56]:
'''sample_hyperparams_list_dict = {
    'initial_lr': [],
    'optimizer': torch.optim.Adam,  # Example optimizer
    'criterion': torch.nn.CrossEntropyLoss(),
    'train_data_used': 0.8,
    'train_set_shuffle': True,
    'train_batch_size': 64
}'''

"sample_hyperparams_list_dict = {\n    'initial_lr': [],\n    'optimizer': torch.optim.Adam,  # Example optimizer\n    'criterion': torch.nn.CrossEntropyLoss(),\n    'train_data_used': 0.8,\n    'train_set_shuffle': True,\n    'train_batch_size': 64\n}"

In [57]:
def generate_hyperparameter_combinations(hyperparams):
    """
    Generate a sequence of hyperparameter combinations.

    :param hyperparams: A dictionary where keys are the names of hyperparameters,
                        and values are lists of possible choices for each hyperparameter.
    :return: A list of dictionaries, each representing a unique combination of hyperparameters.
    """
    # Extract the hyperparameter names and their corresponding choices
    keys, values = zip(*hyperparams.items())

    # Generate all possible combinations of hyperparameter values
    all_combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

    return all_combinations

# Automated Experiments

In [58]:
def get_data_loader(hyperparams, train_dataset, test_dataset):
  shuffle = hyperparams['train_set_shuffle']
  train_batch_size = hyperparams["train_batch_size"]
  train_data_used_num = int(len(train_dataset) * hyperparams["train_data_used"])
  train_indices = torch.randperm(len(train_dataset))[:train_data_used_num]

  train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=train_batch_size, shuffle=shuffle)
  test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)
  return train_loader, test_loader

In [59]:
def run_experiments(hyperparams, train_loader, test_loader, method, num_epochs, num_classes=NUM_CLASSES, in_channels=IN_CHANNELS):
    # Initialize model
    model = Inception(in_channels, num_classes).to(Training_Device)  # Assuming Inception is defined elsewhere

    # Use the optimizer from hyperparameters
    optimizer = hyperparams['optimizer'](model.parameters(), lr=hyperparams['initial_lr'])
    criterion = hyperparams['criterion']

    train_accuracy = {}
    test_accuracy = {}

    for epoch in range(num_epochs):
            # Assuming train and test functions are defined elsewhere
            train_acc = train(epoch, model, train_loader, optimizer, criterion, Training_Device)
            test_acc = test(epoch, model, test_loader, criterion, Training_Device)

            train_accuracy["train accuracy epoch"+str(epoch)] = train_acc
            test_accuracy["test accuracy epoch"+str(epoch)] = test_acc



    if method == "deeplift":
      dl_attributions = print_deeplift(test_loader, model, Feature_Attribution_Device)
      return dl_attributions, train_accuracy, test_accuracy
      print("\n")
    elif method == "integrated_gradients":
      ig_attributions = print_ig(test_loader, model, Feature_Attribution_Device)
      print("\n")
      return ig_attributions, train_accuracy, test_accuracy

# Example Usage
hyperparams = {
    'initial_lr': 0.001,
    'optimizer': torch.optim.Adam,  # Example optimizer
    'criterion': torch.nn.CrossEntropyLoss(),
    'train_data_used': 0.8,
    'train_set_shuffle': True,
    'train_batch_size': 64

}

#train_loader, test_loader = get_data_loader(hyperparams, train_dataset, test_dataset)
#print(run_experiments(num_epochs=1, hyperparams=hyperparams, train_loader=train_loader, test_loader=test_loader))

In [60]:
def run_experiments(hyperparams, train_loader, test_loader, num_epochs, num_classes=NUM_CLASSES, in_channels=IN_CHANNELS):
    # Initialize model
    model = Inception(in_channels, num_classes).to(Training_Device)  # Assuming Inception is defined elsewhere

    # Use the optimizer from hyperparameters
    optimizer = hyperparams['optimizer'](model.parameters(), lr=hyperparams['initial_lr'])
    criterion = hyperparams['criterion']

    train_accuracy = {}
    test_accuracy = {}

    for epoch in range(num_epochs):
            # Assuming train and test functions are defined elsewhere
            train_acc = train(epoch, model, train_loader, optimizer, criterion, Training_Device)
            test_acc = test(epoch, model, test_loader, criterion, Training_Device)

            train_accuracy["train accuracy epoch"+str(epoch)] = train_acc
            test_accuracy["test accuracy epoch"+str(epoch)] = test_acc



    print("deeplift")
    dl_attributions = print_deeplift(test_loader, model, Feature_Attribution_Device)
    print("integrated_gradient")
    ig_attributions = print_ig(test_loader, model, Feature_Attribution_Device)
    return ig_attributions, dl_attributions, train_accuracy, test_accuracy

# Example Usage
hyperparams = {
    'initial_lr': 0.001,
    'optimizer': torch.optim.Adam,  # Example optimizer
    'criterion': torch.nn.CrossEntropyLoss(),
    'train_data_used': 0.8,
    'train_set_shuffle': True,
    'train_batch_size': 64

}

#train_loader, test_loader = get_data_loader(hyperparams, train_dataset, test_dataset)
#print(run_experiments(num_epochs=1, hyperparams=hyperparams, train_loader=train_loader, test_loader=test_loader))

# Functions for saving attribution

In [61]:
def run_experiments_and_save(hyperparams_combinations, train_dataset, test_dataset, csv_file, num_run=NUM_RUN, num_epochs=NUM_EPOCHS):
    """
    Run experiments for each combination of hyperparameters, get feature layer attributions for DeepLift and Integrated Gradients,
    save the results to a CSV file after each run, and skip any combinations that have already been run.

    :param hyperparams_combinations: List of dictionaries with hyperparameter combinations.
    :param train_dataset: Training dataset.
    :param test_dataset: Test dataset.
    :param csv_file: Path to the CSV file for saving results.
    :param num_epochs: Number of epochs for training.
    """

    # Check if the CSV file exists and load existing data
    if os.path.exists(csv_file):
        existing_data = pd.read_csv(csv_file)
    else:
        existing_data = pd.DataFrame()

    for combo in hyperparams_combinations:
        print("\n\n")
        print("Combination:")
        print(combo)
        for i in range(num_run):  # For each run index
            print("runtime" + str(i))
            for method in ['deeplift', 'integrated_gradients']:  # For each method
                print(method)
                # Prepare data for checking if it's already processed
                combo_check = combo.copy()
                combo_check['method'] = method
                combo_check['run'] = i

                # Convert combo_check to string representation
                combo_check = {k: str(v) for k, v in combo_check.items()}



                if not existing_data.empty:
                    # Filter existing_data to only the columns of interest
                    filtered_data = existing_data[list(combo_check.keys())]
                    # Convert filtered_data to string representation
                    filtered_data = filtered_data.astype(str)
                    # Check if a row with the same combination exists
                    if any((filtered_data == pd.Series(combo_check)).all(axis=1)):
                        print("Combination already processed, skipping...")
                        continue  # Skip if combination is already processed

                # Set seed for reproducibility
                random.seed(i)
                np.random.seed(i)
                torch.manual_seed(i)
                if torch.cuda.is_available():
                    torch.cuda.manual_seed_all(i)

                train_loader, test_loader = get_data_loader(combo, train_dataset, test_dataset)

                # Run experiments and compute attributions
                attr = run_experiments(hyperparams=combo, train_loader=train_loader, test_loader=test_loader, method=method, num_epochs=num_epochs)

                # Prepare data for saving
                combo_results = combo.copy()
                for d in attr:
                    combo_results.update(d)

                combo_results['method'] = method
                combo_results['run'] = i

                # Convert all values in combo_results to strings
                combo_results = {k: str(v) for k, v in combo_results.items()}

                # Append results to the existing data
                existing_data = existing_data.append(combo_results, ignore_index=True)

                # Save the data to CSV after each run
                existing_data.to_csv(csv_file, index=False)

In [62]:
def run_experiments_and_save(hyperparams_combinations, train_dataset, test_dataset, csv_file, num_run=NUM_RUN, num_epochs=NUM_EPOCHS):
    """
    Run experiments for each combination of hyperparameters, get feature layer attributions for DeepLift and Integrated Gradients,
    save the results to a CSV file after each run, and skip any combinations that have already been run.

    :param hyperparams_combinations: List of dictionaries with hyperparameter combinations.
    :param train_dataset: Training dataset.
    :param test_dataset: Test dataset.
    :param csv_file: Path to the CSV file for saving results.
    :param num_epochs: Number of epochs for training.
    """

    # Check if the CSV file exists and load existing data
    if os.path.exists(csv_file):
        existing_data = pd.read_csv(csv_file)
    else:
        existing_data = pd.DataFrame()

    for combo in hyperparams_combinations:
        print("\n\n")
        print("Combination:")
        print(combo)
        for i in range(num_run):  # For each run index
            print("runtime" + str(i))
            # Prepare data for checking if it's already processed
            combo_check = combo.copy()
            combo_check['run'] = i

            # Convert combo_check to string representation
            combo_check = {k: str(v) for k, v in combo_check.items()}



            if not existing_data.empty:
                # Filter existing_data to only the columns of interest
                filtered_data = existing_data[list(combo_check.keys())]
                # Convert filtered_data to string representation
                filtered_data = filtered_data.astype(str)
                # Check if a row with the same combination exists
                if any((filtered_data == pd.Series(combo_check)).all(axis=1)):
                    print("Combination already processed, skipping...")
                    continue  # Skip if combination is already processed

                # Set seed for reproducibility
            random.seed(i)
            np.random.seed(i)
            torch.manual_seed(i)
            if torch.cuda.is_available():
                torch.cuda.manual_seed_all(i)

            train_loader, test_loader = get_data_loader(combo, train_dataset, test_dataset)

                # Run experiments and compute attributions
            ig_attributions, dl_attributions, train_accuracy, test_accuracy = run_experiments(hyperparams=combo, train_loader=train_loader, test_loader=test_loader, num_epochs=num_epochs)
            for method in ['deeplift', 'integrated_gradients']:
              if method == 'deeplift':
                  attr = dl_attributions, train_accuracy, test_accuracy
              elif method == 'integrated_gradients':
                  attr = ig_attributions, train_accuracy, test_accuracy

              combo_results = combo.copy()
              for d in attr:
                  combo_results.update(d)

              combo_results['method'] = method
              combo_results['run'] = i

              # Convert all values in combo_results to strings
              combo_results = {k: str(v) for k, v in combo_results.items()}

              # Append results to the existing data
              existing_data = existing_data.append(combo_results, ignore_index=True)

            # Save the data to CSV after each run
            existing_data.to_csv(csv_file, index=False)

# Experiments

In [63]:
hyperparams_choice_list = {
    'initial_lr': [0.001, 0.0007, 0.0003],
    'optimizer': [torch.optim.Adam], # Example optimizer
    'criterion': [torch.nn.CrossEntropyLoss()],
    'train_data_used': [1],
    'train_set_shuffle': [True,False],
    'train_batch_size': [64, 32, 16]
}

combinations = generate_hyperparameter_combinations(hyperparams_choice_list)
for combo in combinations:
    print(combo)

{'initial_lr': 0.001, 'optimizer': <class 'torch.optim.adam.Adam'>, 'criterion': CrossEntropyLoss(), 'train_data_used': 1, 'train_set_shuffle': True, 'train_batch_size': 64}
{'initial_lr': 0.001, 'optimizer': <class 'torch.optim.adam.Adam'>, 'criterion': CrossEntropyLoss(), 'train_data_used': 1, 'train_set_shuffle': True, 'train_batch_size': 32}
{'initial_lr': 0.001, 'optimizer': <class 'torch.optim.adam.Adam'>, 'criterion': CrossEntropyLoss(), 'train_data_used': 1, 'train_set_shuffle': True, 'train_batch_size': 16}
{'initial_lr': 0.001, 'optimizer': <class 'torch.optim.adam.Adam'>, 'criterion': CrossEntropyLoss(), 'train_data_used': 1, 'train_set_shuffle': False, 'train_batch_size': 64}
{'initial_lr': 0.001, 'optimizer': <class 'torch.optim.adam.Adam'>, 'criterion': CrossEntropyLoss(), 'train_data_used': 1, 'train_set_shuffle': False, 'train_batch_size': 32}
{'initial_lr': 0.001, 'optimizer': <class 'torch.optim.adam.Adam'>, 'criterion': CrossEntropyLoss(), 'train_data_used': 1, 'trai

In [64]:
file_path=os.path.join('/content/drive/My Drive/2023 InterpretingNN/code/experiment and result stage2/Baseline fashion', 'Baseline fashion 3*3.csv')
print(file_path)

/content/drive/My Drive/2023 InterpretingNN/code/experiment and result stage2/Baseline fashion/Baseline fashion 3*3.csv


In [65]:
run_experiments_and_save(combinations, train_dataset, test_dataset, csv_file=file_path)




Combination:
{'initial_lr': 0.001, 'optimizer': <class 'torch.optim.adam.Adam'>, 'criterion': CrossEntropyLoss(), 'train_data_used': 1, 'train_set_shuffle': True, 'train_batch_size': 64}
runtime0
Epoch 0: Train Loss = 0.4415, Train Accuracy = 84.66%
Epoch 0: Test Loss = 0.3246, Test Accuracy = 88.09%
deeplift
Layer: branch1.0
157.02527
Layer: branch2.0
162.25171
Layer: branch3.0
146.52983
integrated_gradient
Layer: branch1.0
Attribution: 173.8985993695548
Layer: branch2.0
Attribution: 182.2943392057312
Layer: branch3.0
Attribution: 163.42076526562423
runtime1


<ipython-input-62-566569a8104c>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(combo_results, ignore_index=True)
<ipython-input-62-566569a8104c>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(combo_results, ignore_index=True)


Epoch 0: Train Loss = 0.4186, Train Accuracy = 85.25%
Epoch 0: Test Loss = 0.3613, Test Accuracy = 86.96%
deeplift
Layer: branch1.0
164.20079
Layer: branch2.0
189.58244
Layer: branch3.0
148.97276
integrated_gradient
Layer: branch1.0
Attribution: 172.4409267646976
Layer: branch2.0
Attribution: 199.26450537612422
Layer: branch3.0
Attribution: 155.87178038374165
runtime2


<ipython-input-62-566569a8104c>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(combo_results, ignore_index=True)
<ipython-input-62-566569a8104c>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(combo_results, ignore_index=True)


Epoch 0: Train Loss = 0.4395, Train Accuracy = 85.10%
Epoch 0: Test Loss = 0.3312, Test Accuracy = 87.73%
deeplift
Layer: branch1.0
184.67076
Layer: branch2.0
126.53532
Layer: branch3.0
169.29736
integrated_gradient
Layer: branch1.0
Attribution: 187.76894793020918
Layer: branch2.0
Attribution: 128.7987429896893
Layer: branch3.0
Attribution: 172.4004432553171
runtime3


...

<ipython-input-62-566569a8104c>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(combo_results, ignore_index=True)
<ipython-input-62-566569a8104c>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(combo_results, ignore_index=True)


Epoch 0: Train Loss = 0.3874, Train Accuracy = 85.94%
Epoch 0: Test Loss = 0.3215, Test Accuracy = 88.37%
deeplift
Layer: branch1.0
154.0382
Layer: branch2.0
129.7555
Layer: branch3.0
206.49956
integrated_gradient
Layer: branch1.0
Attribution: 164.0150398364438
Layer: branch2.0
Attribution: 137.27860624001113
Layer: branch3.0
Attribution: 216.31994172159594
runtime14


<ipython-input-62-566569a8104c>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(combo_results, ignore_index=True)
<ipython-input-62-566569a8104c>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(combo_results, ignore_index=True)


Epoch 0: Train Loss = 0.4062, Train Accuracy = 85.33%
Epoch 0: Test Loss = 0.3416, Test Accuracy = 87.69%
deeplift
Layer: branch1.0
145.81358
Layer: branch2.0
170.11745
Layer: branch3.0
174.09427
integrated_gradient
Layer: branch1.0
Attribution: 157.17853441365017
Layer: branch2.0
Attribution: 190.14447916731652
Layer: branch3.0
Attribution: 191.06253520226372


<ipython-input-62-566569a8104c>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(combo_results, ignore_index=True)
<ipython-input-62-566569a8104c>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(combo_results, ignore_index=True)
